In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob as glob

In [2]:
dataset_kk=pd.read_csv('~D:/Vasiliev/PERMAK-A/nv/8d9d/13/y9-1.csv',names=['tvs','tvel','time','kk'],sep=';');
dataset_kk.tvel+=1
dataset_kq=pd.read_csv('~D:/Vasiliev/PERMAK-A/nv/8d9d/13/z9-1.txt',names=['kq']);

cycle_length=26;
state_indexer=0;

dataset_kq['time']=1;
dataset_kq['tvs']=-1;
inx=0;
for _time in range(cycle_length):
    for _tvs in range(163):
        dataset_kq.loc[inx,'time'] = _time;
        dataset_kq.loc[inx,'tvs'] = _tvs+1;
        inx+=1;

        
kqmax=dataset_kq.query('time==0').sort_values('kq').tail(20)
len(np.unique(kqmax.tvs))

20

25.0

In [3]:
def GetDatasetKK(_folder,state_indexer):
    dfa=[];
    for i in _folder:
        #print(i,state_indexer)
        dataset_kk=pd.read_csv(i ,names=['tvs','tvel','time','kk'],sep=';' )
        dataset_kk.tvel+=1
        dataset_kk['state']=state_indexer;
        state_indexer+=1;
        dt_kk=dataset_kk.query('tvel==272 or tvel==282 or tvel==292 or tvel==302 or tvel==312 or tvel==322')
        #dt_kk=dataset_kk.query('tvel==277 or tvel==287 or tvel==297 or tvel==307 or tvel==317 or tvel==327')
        dt_kk=pd.concat([dt_kk.query('tvs==@i') for i in (kqmax.tvs.tolist())], ignore_index=True, sort=False)
        dfa.append(dt_kk);
    df=pd.concat(dfa, ignore_index=True, sort=False);
    print('folder ready!');
    return df,state_indexer;

In [4]:
def GetDatasetKQ(_folder,state_indexer):
    dfa=[]
    for i in _folder:
        #print(i,state_indexer)
        dataset_kq=pd.read_csv(i, names=['kq'])
        dataset_kq['time']=1;
        dataset_kq['tvs']=-1;
        dataset_kq['state']=state_indexer;
        state_indexer+=1;
        inx=0;
        for _time in range(cycle_length):
            for _tvs in range(163):
                dataset_kq.loc[inx,'time'] = _time;
                dataset_kq.loc[inx,'tvs'] = _tvs+1;
                inx+=1;
        dfa.append(dataset_kq);
    df=pd.concat(dfa, ignore_index=True, sort=False);
    print('folder ready');
    return df,state_indexer;

In [5]:
def Dataset(dataset_kq, dataset_kk):
    kr=[]
    kq=[]
    for i in range(len(dataset_kk)):
        val=dataset_kk.iloc[i];
        kq_val=dataset_kq.query('state==@val.state & tvs==@val.tvs')
        kq_val.index=[0]
        kr.append(val.kk*kq_val.kq[0])
        kq.append(kq_val.kq[0])
    dataset_kk['kq']=kq
    dataset_kk['kr']=kr
    print('state ready')
    return dataset_kk;

In [6]:
state_indexer=0;
names_kk= glob.glob('D:/Vasiliev/PERMAK-A/nv/8d9d/13/9/*.csv', recursive=True)
dataset_kk_1,state_indexer=GetDatasetKK(names_kk,state_indexer)
names_kk= glob.glob('D:/Vasiliev/PERMAK-A/nv/8d9d/23/9/*.csv', recursive=True)
dataset_kk_2,state_indexer=GetDatasetKK(names_kk,state_indexer)
names_kk= glob.glob('D:/Vasiliev/PERMAK-A/nv/8d9d/33/9/*.csv', recursive=True)
dataset_kk_3,state_indexer=GetDatasetKK(names_kk,state_indexer)
# Merging dataset with kk
cdf=[dataset_kk_1,dataset_kk_2,dataset_kk_3];
dataset_kr=pd.concat(cdf, ignore_index=True, sort=False)

folder ready!

folder ready!

folder ready!



Получили все параметры для угловых твэлов из ТВС в kqmax 

In [7]:
state_indexer=0;
names_kq= glob.glob('D:/Vasiliev/PERMAK-A/nv/8d9d/13/9/*.txt', recursive=True)
dataset_kq_1,state_indexer=GetDatasetKQ(names_kq, state_indexer)
names_kq= glob.glob('D:/Vasiliev/PERMAK-A/nv/8d9d/23/9/*.txt', recursive=True)
dataset_kq_2,state_indexer=GetDatasetKQ(names_kq,state_indexer)
names_kq= glob.glob('D:/Vasiliev/PERMAK-A/nv/8d9d/33/9/*.txt', recursive=True)
dataset_kq_3,state_indexer=GetDatasetKQ(names_kq,state_indexer)
dataset_kq_u=pd.concat([dataset_kq_1,dataset_kq_2,dataset_kq_3],ignore_index=True, sort=False);
dataset_kq=pd.concat([dataset_kq_u.query('tvs==@i') for i in (kqmax.tvs)], ignore_index=True, sort=False)

folder ready
folder ready
folder ready


In [8]:
united_b=Dataset(dataset_kq.query('time==0'), dataset_kr.query('time==0'))

state ready


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [9]:
united_m=Dataset(dataset_kq.query('time==13'), dataset_kr.query('time==13'))

state ready


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [10]:
united_e=Dataset(dataset_kq.query('time==25'), dataset_kr.query('time==25'))

state ready


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [11]:
united_b.to_csv('D:/Vasiliev/PERMAK-A/nv/8d9d/cen_time0_b.csv', index=False)
united_m.to_csv('D:/Vasiliev/PERMAK-A/nv/8d9d/cen_time0_m.csv',index=False)
united_e.to_csv('D:/Vasiliev/PERMAK-A/nv/8d9d/cen_time0_e.csv',index=False)